In [206]:
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [207]:
%pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# api_key = "pinecone_key"

In [214]:
from langchain_openai import OpenAIEmbeddings

In [215]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [244]:
import os
from pinecone import Pinecone , ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
# initialize the pinecone client
pc = Pinecone(api_key = api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = "chatbot",
        dimension = 384,  # dimentions of dense vector
        metric= 'dotproduct' , # for sparse values
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )

In [238]:
index = pc.Index(index_name)
index

In [ ]:
# os.environ["OPENAI_API_KEY"] = "openAi_key"
 
# os.environ["HF_TOKEN"] = "huggingFace_key"

In [245]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()
import os

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model= 'text-embedding-3-large')
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001E349838FB0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001E342EC6DE0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [220]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [249]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = "data.csv"

loader = CSVLoader(file_path=file_path)
data = loader.load()

print(type(data[0].page_content))
text_list = [doc.page_content for doc in data]

<class 'str'>


In [222]:
## tfidf values on these sentences 
bm25_encoder.fit(text_list)

## store the values to json file 
bm25_encoder.dump("bm25_values.json")   

## loading to BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 1000/1000 [00:03<00:00, 316.62it/s]


In [250]:
retriever =PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index,top_k=10)

In [247]:
retriever

PineconeHybridSearchRetriever(embeddings=OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001E349838FB0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001E342EC6DE0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001E33D6AE0C0>, index=<pinecone.data.index.Ind

In [ ]:
retriever.add_texts(text_list)

In [121]:
result = retriever.invoke("sku : 47-10-291-55, price: 2.93 ,captive screws")

In [122]:
print(type(result))

<class 'list'>


In [156]:
retriever.invoke("what is difference between price of sku 47-10-291-55 and E3-124-105")

[Document(metadata={'score': 0.225731224}, page_content='R4-0-60784 0.41 Available in seven different options and end fitting styles to suit a wide range of application requirements, Southco?? AC Rotary Latch Cables connect our line of actuators and rotary latches for remote actuation. Constructed of high-strength, corrosion-resistant materials, Southco?? AC Rotary Latch Cables are suitable for use in both interior and exterior environments Latch nan Micro Acetal Copolymer nan Industrial Hardware Compression Latches, Industrial Latches Plastics nan nan Micro Size Interior and Exterior Environments nan nan The Rotary Push-to-Close Latch Striker, Micro Size, Acetal Copolymer 0.41 is a versatile latching solution available in seven different options and end fitting styles to suit a wide range of application requirements. It is suitable for use in both interior and exterior environments. Priced at 0.41, this latch is an excellent choice for professionals seeking long-lasting and effective 

In [184]:
retriever.invoke("what is difference between description of sku 47-10-291-55 and E3-124-105")

[Document(metadata={'score': 0.226833403}, page_content='47-10-291-55 2.74 Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications. captive screw 6-32 floating unknown smooth black industrial hardware captive screws, limited-space applications unknown smooth unknown unknown doors, panels, industrial equipment unknown unknown'),
 Document(metadata={'score': 0.217884585}, page_content='R4-0-60784 0.41 Available in seven different options and end fitting styles to suit a wide range of application requirements, Southco?? AC Rotary Latch Cables connect our line of actuators and rotary latches for remote actuation. Constructed of high-strength, corrosion-resistant materials, Southco?? AC Rotary Latch Cables are suitable for use in both interior and exterior environments Latch nan Micro Ace

In [69]:
retriever.invoke("81-35-308-5")

[Document(metadata={'score': 0.150857493}, page_content='47-65-281-30 2.74 Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications. captive screw 6-32 captive screw unknown bright industrial hardware captive screws, limited-space applications unknown bright finish unknown unknown doors, panels, industrial equipment unknown unknown'),
 Document(metadata={'score': 0.149385393}, page_content='47-12-224-22 7.31 Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications. captive screw 6-32 flare-in unknown bright industrial hardware captive screws, limited-space applications unknown bright

In [158]:

retriever.invoke("zk-25-912-10")

[Document(metadata={'score': 0.186279878}, page_content='id: 2037\nsku: E5-1-005-020\nname: Quarter-Turn Cam Latch, Slotted Recess, 2mm (.08 in) Grip, Not Sealed, Zinc Alloy, Powder Coat, Black\nprice: 10.41\ndescription: These latches deliver affordable simplicity, including quick access with just a quarter turn, the versatility of multiple body materials, plus the flexibility of modular design for numerous actuation and dimensional options. Ramped cams provide a snug fit to minimize vibration and rattle\nimage: /4/5/453823959_importvideo_4752.jpg\nproduct_type: Quarter-Turn Cam Latch\nthread_size: \nshape: \nmaterial: Zinc Alloy\nfinish: Powder Coat\nproduct_category: industrial hardware\nsubcategories: compression latches, industrial latches\nmaterial_category: metals, alloys\nfinish_type: Powder Coat\ngrip_type: \nsize_specifications: 2mm (.08 in) Grip\napplication_area: \ninstallation_method: \ncompliance_standards: \nlengthy_text: The Quarter-Turn Cam Latch, Slotted Recess, 2mm (

In [70]:
retriever.invoke("what is the price of 81-35-308-55")

[Document(metadata={'score': 0.353918076}, page_content='81-35-308-55 3.8 The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements. quarter turn fasteners nan nan steel zinc plate bright chromate industrial hardware mounting requirements metals bright finish nan small size panels quarter turn nan The DZUS? Lion Quarter-Turn Receptacle, Small Size, Steel Zinc Plate, Bright chromate 3.8 is a versatile fastening solution designed for securing panels quickly and easily. With numerous head styles, stud lengths, receptacle styles, and accessories available, it provides flexibility for accommodating various mounting requirements. Priced at 3.8, this receptacle is an excellent choice for professionals seeking efficient and reliable hardware solutions for industrial applications.'),
 D

In [72]:
retriever.invoke("what is the description of E5-1-001-031")

[Document(metadata={'score': 0.460495412}, page_content='E5-1-001-031 11.64 These latches deliver affordable simplicity, including quick access with just a quarter turn, the versatility of multiple body materials, plus the flexibility of modular design for numerous actuation and dimensional options. Ramped cams provide a snug fit to minimize vibration and rattle Quarter-Turn Cam Latch 4mm (.16 in) Slotted Recess Zinc Alloy Chrome Plated industrial hardware compression latches, industrial latches metals, alloys Bright Sealed 4mm (.16 in) Grip doors, panels, industrial equipment quarter turn nan'),
 Document(metadata={'score': 0.415606916}, page_content='E5-1-001-031 11.64 These latches deliver affordable simplicity, including quick access with just a quarter turn, the versatility of multiple body materials, plus the flexibility of modular design for numerous actuation and dimensional options. Ramped cams provide a snug fit to minimize vibration and rattle Quarter-Turn Cam Latch 4mm (.16

# test

In [161]:
answers = []

In [ ]:
answers.append(retriever.invoke("tell me something about 81-46-101-39."))

[Document(metadata={'score': 0.132393658}, page_content='id: 7\nsku: 81-46-101-39\nname: DZUS? Lion Quarter-Turn Accessory, Small Size, Plastic Natural\nprice: 0.33\ndescription: The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements.\nimage: /8/1/81-46-101-39-1000x1000_1.jpg\nproduct_type: quarter turn\nthread_size: \nshape: \nmaterial: plastic\nfinish: \nproduct_category: industrial hardware\nsubcategories: accessories, fasteners\nmaterial_category: plastics\nfinish_type: \ngrip_type: \nsize_specifications: small size\napplication_area: panels\ninstallation_method: quarter turn\ncompliance_standards: \nlengthy_text: The DZUS? Lion Quarter-Turn Accessory, Small Size, Plastic Natural 0.33 is a versatile fastening solution designed for quick and easy panel securing. It featur

In [162]:
answers.append(retriever.invoke("tell me something about 81-46-101-39."))

In [163]:
retriever.invoke("what is the price of 81-32-301-12")

[Document(metadata={'score': 0.347154558}, page_content='id: 5\nsku: 81-32-301-12\nname: DZUS? Lion Quarter-Turn Retainer, Small Size, Plastic\nprice: 0.17\ndescription: The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements.\nimage: /8/1/81-32-301-12-1000x1000_1.jpg\nproduct_type: Retainer\nthread_size: \nshape: \nmaterial: Plastic\nfinish: \nproduct_category: industrial hardware\nsubcategories: compression latches, industrial latches\nmaterial_category: plastics\nfinish_type: \ngrip_type: \nsize_specifications: Small Size\napplication_area: panels\ninstallation_method: quarter turn\ncompliance_standards: \nlengthy_text: The DZUS? Lion Quarter-Turn Retainer, Small Size, Plastic 0.17 is a versatile fastener designed for securing panels quickly and easily. It features numerou

In [164]:
answers.append(retriever.invoke("what is the price of 81-32-301-12"))

In [165]:
retriever.invoke("give me the sku of this Key, Bellcore Electronics, Zinc Alloy, Nickel Plated")

[Document(metadata={'score': 0.384964436}, page_content='id: 291571\nsku: E5-8-132-UU1\nname: Quarter-Turn Cam Latch,  Bellcore 216 Shuttered Recess, Zinc Alloy, Chrome Plated, Bright\nprice: 13.97\ndescription: These latches deliver affordable simplicity, including quick access with just a quarter turn, the versatility of multiple body materials, plus the flexibility of modular design for numerous actuation and dimensional options. Ramped cams provide a snug fit to minimize vibration and rattle\nimage: /4/5/453828206_importvideo_2_3528.jpg\nproduct_type: Quarter-Turn Cam Latch\nthread_size: \nshape: \nmaterial: Zinc Alloy\nfinish: Chrome Plated\nproduct_category: industrial hardware\nsubcategories: compression latches, industrial latches\nmaterial_category: metals, alloys\nfinish_type: bright finish\ngrip_type: \nsize_specifications: \napplication_area: doors, panels, industrial equipment\ninstallation_method: \ncompliance_standards: \nlengthy_text: The Quarter-Turn Cam Latch, Bellcor

In [166]:
answers.append(retriever.invoke("give me the sku of this Key, Bellcore Electronics, Zinc Alloy, Nickel Plated"))

In [167]:

retriever.invoke("tell me about this Flush Mounting Bracket, Steel, Galvanized in 2 line. and sell me this.")

[Document(metadata={'score': 0.320532262}, page_content='id: 253\nsku: 82-19-100-20\nname: DZUS? Lion Quarter-Turn Stud, Medium Size, Phillips, Flush, Stainless Steel Passivated\nprice: 2\ndescription: The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements.\nimage: /8/2/82-19-240-16-1000x1000_2.jpg\nproduct_type: quarter turn stud\nthread_size: \nshape: \nmaterial: stainless steel\nfinish: flush\nproduct_category: industrial hardware\nsubcategories: compression latches, industrial latches\nmaterial_category: metals\nfinish_type: \ngrip_type: \nsize_specifications: medium size\napplication_area: panels\ninstallation_method: quarter turn\ncompliance_standards: \nlengthy_text: The DZUS? Lion Quarter-Turn Stud, Medium Size, Phillips, Flush, Stainless Steel Passivated 2.0 is a ve

In [168]:
answers.append(retriever.invoke("tell me about this Flush Mounting Bracket, Steel, Galvanized in 2 line. and sell me this."))

In [169]:
retriever.invoke("what is the purpose of using DZUS? Lion Quarter-Turn Retainer, Small Size, Plastic. also mention its sku.")

[Document(metadata={'score': 0.500771}, page_content='The DZUS Lion Quarter-Turn Retainer, Medium Size, Plastic 0.13 is a versatile fastener designed for securing panels quickly and easily. It features numerous head styles, stud lengths, receptacle styles, and accessories to accommodate various mounting requirements. Priced at 0.13, this retainer is an excellent choice for professionals seeking efficient and effective hardware solutions for industrial applications.'),
 Document(metadata={'score': 0.483511329}, page_content='The DZUS Lion Quarter-Turn Retainer, Medium Size, Plastic 0.13 is a versatile fastener designed for securing panels quickly and easily. It features numerous head styles, stud lengths, receptacle styles, and accessories to accommodate various mounting requirements. Priced at 0.13, this retainer is an excellent choice for professionals seeking efficient and effective hardware solutions for industrial applications. 82-32-301-12'),
 Document(metadata={'score': 0.4607414

In [170]:
answers.append(retriever.invoke("what is the purpose of using DZUS? Lion Quarter-Turn Retainer, Small Size, Plastic. also mention its sku."))

In [171]:
retriever.invoke("ok what is the price of this.")

[Document(metadata={'score': 0.150247917}, page_content='id: 100\nsku: 47-10-121-11\nname: Captive Screw, 4-40 Thread, Flare-in, Phillips Recess, Smooth Bright Knob\nprice: 2.61\ndescription: Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications.\nimage: /4/7/47-90-321-25_1080.jpg\nproduct_type: captive screw\nthread_size: 4-40\nshape: flare-in\nmaterial: smooth bright knob\nfinish: smooth bright\nproduct_category: industrial hardware\nsubcategories: captive screws, prism 4c class captive screws\nmaterial_category: metals\nfinish_type: smooth bright finish\ngrip_type: \nsize_specifications: \napplication_area: doors, panels, limited-space applications\ninstallation_method: \ncompliance_standards: \nlengthy_text: The Captive Screw, 4-40 Thread, Flare-in, Phillips Recess, Smooth Brig

In [172]:
answers.append(retriever.invoke("ok what is the price of this."))

In [173]:
retriever.invoke("what is DZUS Lion Quarter-Turn Receptacle, Medium Size.")

[Document(metadata={'score': 0.683132768}, page_content='id: 13\nsku: 82-46-101-39\nname: DZUS? Lion Quarter-Turn Accessory, Medium Size, Plastic Natural\nprice: 0.28\ndescription: The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements.\nimage: /8/1/81-46-101-39-1000x1000_3.jpg\nproduct_type: quarter turn\nthread_size: \nshape: \nmaterial: plastic\nfinish: \nproduct_category: industrial hardware\nsubcategories: accessories, fasteners\nmaterial_category: plastics\nfinish_type: \ngrip_type: \nsize_specifications: medium size\napplication_area: panels\ninstallation_method: quarter turn\ncompliance_standards: \nlengthy_text: The DZUS Lion Quarter-Turn Accessory, Medium Size, Plastic Natural 0.28 is a versatile fastening solution designed for quick and easy panel securing. With n

In [174]:
answers.append(retriever.invoke("what is DZUS Lion Quarter-Turn Receptacle, Medium Size."))

In [175]:
print(answers)

[[Document(metadata={'score': 0.132393658}, page_content='id: 7\nsku: 81-46-101-39\nname: DZUS? Lion Quarter-Turn Accessory, Small Size, Plastic Natural\nprice: 0.33\ndescription: The simplicity of a quarter turn is all it takes to secure panels quickly and easily with these versatile little fasteners. Numerous head styles, stud lengths, receptacle styles, and accessories, provide flexibility for accommodating many specific mounting requirements.\nimage: /8/1/81-46-101-39-1000x1000_1.jpg\nproduct_type: quarter turn\nthread_size: \nshape: \nmaterial: plastic\nfinish: \nproduct_category: industrial hardware\nsubcategories: accessories, fasteners\nmaterial_category: plastics\nfinish_type: \ngrip_type: \nsize_specifications: small size\napplication_area: panels\ninstallation_method: quarter turn\ncompliance_standards: \nlengthy_text: The DZUS? Lion Quarter-Turn Accessory, Small Size, Plastic Natural 0.33 is a versatile fastening solution designed for quick and easy panel securing. It featu

In [183]:
retriever.invoke("Alternative of 47-11-391-10")

[Document(metadata={'score': 0.269513667}, page_content='47-11-391-10 2.91 Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications. captive screw 8-32 floating unknown bright industrial hardware captive screws, limited-space applications unknown bright finish knurled 2.91 doors, panels, industrial equipment unknown unknown'),
 Document(metadata={'score': 0.229797244}, page_content='47-11-391-10 2.91 Southco 47 Class captive screws (and the corresponding color-coded "Prism" 4C Class captive screws), offer hundreds of permutations of installation and tool-actuation styles for mounting doors, panels, and other hardware components in limited-space applications. captive screw 8-32 floating unknown bright industrial hardware captive screws, limited-space applications unknown bright finish 